In [1]:
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import pandas as pd

In [2]:
from datetime import datetime
import locale
locale.setlocale(locale.LC_ALL, '')

def replace_month(month_gen):
    if month_gen == 'мая':
        return 'май'
    else:
        return month_gen[:3]

def parse_ru_datetime(s):
    s = s.split()
    s[1] = replace_month(s[1])
    s = ' '.join(s)
    return datetime.strptime(s, '%d %b %Y | %H:%M')

In [3]:
def clean_text(text):
    return text.replace('\n', ' ').replace('\xa0', ' ')

In [4]:
def parse_score(s):
    try:
        return int(next(re.finditer('\d+ из \d+', s))[0].split()[0])
    except:
        return 0

In [78]:
s = 'кплщкопшщуокп выалывжа вкпщшокщп вчмшгор98ыуе цуп из ывашщывоащ 3 из 10 ывджалвыж из 10'

In [72]:
s = 'sdfsdfdsf'

In [81]:
parse_score(s)

3

In [8]:
%pwd

'C:\\Users\\Sergey\\Desktop\\Лихарев Сергей\\МАИ\\Информатика\\AI\\AI\\kinopoisk\\good'

In [49]:
%cd ../neutral

C:\Users\Sergey\Desktop\Лихарев Сергей\МАИ\Информатика\AI\kinopoisk\neutral


In [50]:
files = ! dir /B .
print(len(files))

8


In [52]:
import io
def parse_info(review):
    d = dict()
    d['movie_title']        = review.select('p.film span')[0].text
    try:
        d['date']  = parse_ru_datetime(
                            review.select('span.date')[0].text)
    except:
        return None
    tmp = review.select('ul.useful li')[2].text.split(' / ')
    d['useful']       = int(tmp[0])
    d['not_useful']   = int(tmp[1])
    d['title'] = clean_text(review.select('p.sub_title')[0].text)
    d['text']  = clean_text(review.find('span', itemprop='reviewBody').text)
    d['score'] = parse_score(d['text'])
    return d


list_of_reviews = []
for file in files:
    with io.open(file, encoding='utf-8', errors='ignore') as f:
        content = f.read()
        soup = BeautifulSoup(content, 'html.parser')
        for review in tqdm(soup.select('div.response.bad')[2:]):
            d = parse_info(review)
            if d:
                d['cat'] = 'bad'
                list_of_reviews.append(d)
        for review in tqdm(soup.select('div.response.good')[2:]):
            d = parse_info(review)
            if d:
                d['cat'] = 'good'
                list_of_reviews.append(d)
        for review in tqdm(soup.select('div.response')[2:]):
            d = parse_info(review)
            if d:
                d['cat'] = 'neutral'
                list_of_reviews.append(d)
    #             print(review.text)

0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████████████████████████████████████████████████████████████████████████| 198/198 [00:00<00:00, 1090.64it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████████████████████████████████████████████████████████████████████████| 198/198 [00:00<00:00, 1102.91it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████████████████████████████████████████████████████████████████████████| 122/122 [00:00<00:00, 1019.39it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████████████████████████████████████████████████████████████████████████| 198/198 [00:00<00:00, 1123.27it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████████████████████████████████████████████████████████████████████████| 198/198 [00:00<00:00, 1061.65it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|█████████████████████████████████████████████████████████████████████████████████| 33/33 [00:00<00:00, 807.09it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|█████████

In [53]:
df = pd.DataFrame(list_of_reviews)
df.sort_values('date').tail()

,cat,date,movie_title,not_useful,score,text,title,useful
950,neutral,2019-06-05 21:57:00,Во имя спасения,0,6,Апокалипсис по шведски. За крепкий и перегруже...,Замечательная бредятина.,2
949,neutral,2019-06-06 00:46:00,Атлантида,0,6,Научно-фантастический фильм «Атлантида»  симб...,,0
948,neutral,2019-06-06 11:37:00,Мы всегда жили в замке,0,5,«Мы всегда жили в замке»  второй фильм Стэйси...,,0
947,neutral,2019-06-06 14:03:00,Годзилла 2: Король монстров,0,6,"Начну с того, что не являюсь фанатом Годзиллы,...",Король спецэффектов,0
946,neutral,2019-06-06 14:55:00,Люди Икс: Тёмный Феникс,0,6,"Сюжет  предсказуемый, шаблонный, угадывается ...",Не так должна была закончиться культовая сага.,0


In [54]:
df.to_csv('../neutral.tsv', sep='\t')